<a href="https://colab.research.google.com/github/elichen/aoc2024/blob/main/Day_24_Crossed_Wires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
input = """x00: 1
x01: 0
x02: 1
x03: 1
x04: 0
y00: 1
y01: 1
y02: 1
y03: 1
y04: 1

ntg XOR fgs -> mjb
y02 OR x01 -> tnw
kwq OR kpj -> z05
x00 OR x03 -> fst
tgd XOR rvg -> z01
vdt OR tnw -> bfw
bfw AND frj -> z10
ffh OR nrd -> bqk
y00 AND y03 -> djm
y03 OR y00 -> psh
bqk OR frj -> z08
tnw OR fst -> frj
gnj AND tgd -> z11
bfw XOR mjb -> z00
x03 OR x00 -> vdt
gnj AND wpb -> z02
x04 AND y00 -> kjc
djm OR pbm -> qhw
nrd AND vdt -> hwm
kjc AND fst -> rvg
y04 OR y02 -> fgs
y01 AND x02 -> pbm
ntg OR kjc -> kwq
psh XOR fgs -> tgd
qhw XOR tgd -> z09
pbm OR djm -> kpj
x03 XOR y03 -> ffh
x00 XOR y04 -> ntg
bfw OR bqk -> z06
nrd XOR fgs -> wpb
frj XOR qhw -> z04
bqk OR frj -> z07
y03 OR x01 -> nrd
hwm AND bqk -> z03
tgd XOR rvg -> z12
tnw OR pbm -> gnj"""

In [52]:
input = open("input.txt").read()

In [53]:
def parse_input(input_str):
    lines = input_str.strip().split('\n')

    # Parse wire values
    wire_values = {}
    dependencies = {}

    # Find the empty line that separates the two parts
    split_idx = lines.index('')

    # Parse wire values
    for line in lines[:split_idx]:
        wire, value = line.split(': ')
        wire_values[wire] = int(value)

    # Parse dependencies
    for line in lines[split_idx+1:]:
        left, right = line.split(' -> ')
        inputs = left.split(' ')
        dependencies[right] = (inputs[0], inputs[1], inputs[2])  # (wire1, operator, wire2)

    return wire_values, dependencies

def evaluate(wire, wire_values, dependencies):
    # Base case: if wire value is known
    if wire in wire_values:
        return wire_values[wire]

    # Get dependency info
    wire1, op, wire2 = dependencies[wire]

    # Recursively evaluate input wires
    val1 = evaluate(wire1, wire_values, dependencies)
    val2 = evaluate(wire2, wire_values, dependencies)

    # Compute result based on operator
    if op == 'AND':
        result = val1 & val2
    elif op == 'OR':
        result = val1 | val2
    elif op == 'XOR':
        result = val1 ^ val2

    # Cache result
    wire_values[wire] = result
    return result

def solve_circuit(input_str):
    wire_values, dependencies = parse_input(input_str)

    # Get all z wires and sort them
    z_wires = sorted([wire for wire in dependencies.keys() if wire.startswith('z')])

    # Evaluate each z wire
    results = {}
    for z_wire in z_wires:
        results[z_wire] = evaluate(z_wire, wire_values, dependencies)

    return results


result = solve_circuit(input)
sum(v << i for i, (k, v) in enumerate(sorted(result.items())))

61886126253040

In [174]:
def print_gate_tree(gate, dependencies, indent="", is_last=True, depth=None, current_depth=0):
    # If we've reached max depth, stop here
    if depth is not None and current_depth >= depth:
        return [indent + ("└── " if is_last else "├── ") + "..."]

    if gate not in dependencies:
        return [indent + ("└── " if is_last else "├── ") + gate]

    wire1, op, wire2 = dependencies[gate]

    # Create current line
    current = indent + ("└── " if is_last else "├── ") + gate

    # Create new indent for children
    new_indent = indent + ("    " if is_last else "│   ")

    # Create operation line
    op_line = new_indent + "└── " + op

    # Get children's lines with incremented depth
    wire1_lines = print_gate_tree(wire1, dependencies, new_indent + "    ", False, depth, current_depth + 1)
    wire2_lines = print_gate_tree(wire2, dependencies, new_indent + "    ", True, depth, current_depth + 1)

    return [current] + [op_line] + wire1_lines + wire2_lines

# z -> xor -> or -> and
#                -> and
#          -> xor
#                -> x,y

swaps = [("z07","nqk"),("pcp","fgt"),("z24","fpq"),("z32","srn")]
wires, gates = parse_input(input)
for pair in swaps:
  swap_gates(gates, *pair)
z_gates = sorted(gate for gate in gates if gate.startswith('z'))
for gate in z_gates:
  print("\n".join(print_gate_tree(gate,gates, depth=5)))

└── z00
    └── XOR
        ├── y00
        └── x00
└── z01
    └── XOR
        ├── hfm
        │   └── AND
        │       ├── y00
        │       └── x00
        └── hqt
            └── XOR
                ├── y01
                └── x01
└── z02
    └── XOR
        ├── dmw
        │   └── OR
        │       ├── qng
        │       │   └── AND
        │       │       ├── x01
        │       │       └── y01
        │       └── hkm
        │           └── AND
        │               ├── hqt
        │               │   └── XOR
        │               │       ├── y01
        │               │       └── x01
        │               └── hfm
        │                   └── AND
        │                       ├── y00
        │                       └── x00
        └── bfr
            └── XOR
                ├── y02
                └── x02
└── z03
    └── XOR
        ├── wmn
        │   └── XOR
        │       ├── x03
        │       └── y03
        └── rmb
            └── OR
                ├─

In [182]:
",".join(sorted(sum(([a, b] for a, b in swaps), [])))

'fgt,fpq,nqk,pcp,srn,z07,z24,z32'